In [2]:
import os
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_experimental.text_splitter import SemanticChunker


In [ ]:
EMBEDDING_MODEL_NAME="BAAI/bge-small-en"

In [4]:
loader = DirectoryLoader(
    os.path.abspath('./data'),
    glob="**/*.pdf", 
    # use_multithreading=True,
    show_progress=True,
    max_concurrency=50,
    loader_cls=UnstructuredPDFLoader,
    sample_size=2
)


In [5]:
def get_embeddings():
    model_kwargs = {"device": "cuda"}
    encode_kwargs = {"normalize_embeddings": True}
    embeddings = HuggingFaceBgeEmbeddings(
        model_name=EMBEDDING_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
    )
    return embeddings

In [ ]:
embeddings = get_embeddings()
text_splitter = SemanticChunker(
    embeddings=embeddings
)
